In [44]:
#pip install geopy

In [45]:
#pip install geopandas

In [46]:
#pip install xlrd

In [47]:
import pandas as pd
import numpy as np
import json
from geopy.distance import geodesic 
import geopandas as gpd
import math

In [48]:
df = pd.read_csv("Blocks Data - DD_sample_data.csv")

In [49]:
df.shape

(2127, 71)

In [50]:
df_zips = pd.read_excel("ZIP_Locale_Detail.xls")
#https://postalpro.usps.com/ZIP_Locale_Detail

In [51]:
#import the county-zip mapping file (US Census)
df_county = gpd.read_file("PVS_23_v2_ccd_12011.shp")

Streamline Zip Codes for location mapping

In [52]:
#make sure all zips are 5 digit strings
def zips(i):
    y = str(i)
    if len(y) == 3:
        x = ''.join(("00",y))
    elif len(y) == 4:
        x = ''.join(("0",y))
    else:
        x = y
    return x

In [53]:
df_zips["PHYSICAL ZIP_CHANGE"] = df_zips["PHYSICAL ZIP"].apply(lambda x: zips(x))

In [54]:
#convert extras from string to dictionary
df['extras'] = df['extras'].apply(lambda x: json.loads(x))

Create location fields for mapping and calculate distance between clean voting and collection location fields

In [55]:
def concat(x,y):
    if pd.isna(x):
        x = 0
    if pd.isna(y):
        y = 0
    return f"{x},{y}"

In [56]:
df['voting_location'] = df.apply(lambda row: concat(row['voting_address_latitude'], row['voting_address_longitude']), axis=1)

In [57]:
df['collection_location'] = df.apply(lambda row: concat(row['collection_location_latitude'], row['collection_location_longitude']), axis=1)

In [58]:
def dist(x, y):
    return round(geodesic(eval(x),eval(y)).miles,1)


In [59]:
#extract dictionary into df with separate columns for each key
df_extras = df.extras.apply(pd.Series)

In [60]:
#merge original df with dictionary df
df_clean = pd.merge(right=df, left=df_extras, right_index = True, left_index=True)

In [111]:
#merge with streamlined zip codes for voting location
df_clean_location = pd.merge(left=df_clean, right=df_zips[["PHYSICAL ZIP", "PHYSICAL CITY"]].drop_duplicates(subset=['PHYSICAL ZIP']), left_on = "voting_zipcode", right_on = "PHYSICAL ZIP",how='left')

In [113]:
df_clean_location.rename(columns={"PHYSICAL CITY": "voting_city_clean"}, inplace=True)

In [114]:
df_clean_location = df_clean_location.drop(["PHYSICAL ZIP"], axis=1)

In [132]:
#merge with streamlined zip codes for collection location
df_clean_location_ = pd.merge(left=df_clean_location, right=df_zips[["PHYSICAL ZIP", "PHYSICAL CITY"]].drop_duplicates(subset=['PHYSICAL ZIP']), left_on = "collection_location_zip", right_on = "PHYSICAL ZIP", how="left")

In [134]:
df_clean_location_.rename(columns={"PHYSICAL CITY": "collection_city_clean"}, inplace=True)

In [135]:
df_clean_location_["distance"] = df_clean_location_.apply(lambda row: dist(row['voting_location'], row['collection_location']), axis=1)

Clean up disparate versions for all dem fields extracted from Extras

In [67]:
#combine gender columns
gender_columns = [col for col in df_clean_location_.columns if 'gender' in col]

In [136]:
df_clean_location_['gender_combined'] = df_clean_location_[gender_columns].bfill(axis=1).iloc[:, 0]

In [69]:
#combine race columns
race_columns = [col for col in df_clean_location_.columns if 'race' in col]

In [137]:
df_clean_location_['race_combined']=df_clean_location_[race_columns].bfill(axis=1).iloc[:, 0]

In [71]:
#combine birth location columns
birth_combined = [col for col in df_clean_location_.columns if 'birth' in col]

In [138]:
df_clean_location_['birthplace_combined']=df_clean_location_[birth_combined].bfill(axis=1).iloc[:, 0]

In [139]:
df_clean_location_['gender']=df_clean_location_['gender_combined']
df_clean_location_['race and ethnicity'] = df_clean_location_['race_combined']
df_clean_location_['birthplace'] = df_clean_location_['birthplace_combined']
df_clean_location_.drop(columns=['gender_combined', 'race_combined','birthplace_combined'], inplace=True)

Clean up County Columns

In [74]:
#create a clean lookup file (Zip Codes to Counties)
df_zips_counties = pd.read_csv("Zips and Counties - Sheet1.csv", header=None, names=["data"])

In [75]:
df_zips_counties['zips_filter']=df_zips_counties['data'].str.isnumeric()
df_zips_counties['counties_filter']=df_zips_counties['data'].str.contains(" County")

zips = df_zips_counties[df_zips_counties['zips_filter']==True]
counties = df_zips_counties[df_zips_counties['counties_filter']==True]


In [76]:
# combine zips and counties 
zips.reset_index(drop=True, inplace=True)
counties.reset_index(drop=True, inplace=True)
df_zips_counties_combined = pd.concat([zips,counties], axis=1)

In [77]:
df_zips_counties_combined.columns.values[0]='zip_code'
df_zips_counties_combined.columns.values[3]='county'

In [78]:
df_zips_counties_combined['zip_code']=df_zips_counties_combined['zip_code'].astype(float)

In [143]:
#apply clean zip code file to full dataset
df_clean_location__ = pd.merge(left=df_clean_location_, right=df_zips_counties_combined, left_on = "voting_zipcode", right_on = "zip_code",how='left')

In [145]:
df_clean_location__.rename(columns={"county": "voting_county"}, inplace=True)

In [146]:
df_clean_location__['voting_street_address_one'] = df_clean_location__['voting_county']

df_clean_location__.drop(columns=[
  'zip_code', 'zips_filter',
       'counties_filter', 'voting_county', 'zips_filter', 'counties_filter'], inplace=True)

In [147]:
#create csv
df_clean_location__.to_csv("clean_blocks_.csv", index=False)